In [ ]:
import os

import numpy as np
import pandas as pd

import matplotlib.ticker as plticker
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import matplotlib as mpl
import seaborn as sns

from sklearn.metrics import roc_curve, auc

from rdkit.Chem import AllChem as Chem
from rdkit import DataStructs

import warnings
warnings.filterwarnings("ignore")

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams["figure.dpi"] = 300
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = ['Arial']

In [ ]:
#LBVS
def read_smis(smis_path):
    with open(smis_path) as f:
        smis = [l.rstrip() for l in f]
    return smis

def Compute_RocAuc(ligands, decoys, fptype="ECFP4"):   
    S_Ligand = read_smis(ligands)
    length_Ligand = len(S_Ligand)
    Suppl_Ligand = []
    for i in range(0,length_Ligand):
        m = Chem.MolFromSmiles(S_Ligand[i])
        if m:
            Suppl_Ligand.append(m)

    S_Decoy = read_smis(decoys)
    length_Decoy= len(S_Decoy)
    Suppl_Decoy = []
    for i in range(0,length_Decoy):
        m = Chem.MolFromSmiles(S_Decoy[i])
        if m:
            Suppl_Decoy.append(m)

    if fptype == "ECFP4":
        Fps_Ligand = [Chem.GetMorganFingerprintAsBitVect(x, 2, nBits=1024) for x in Suppl_Ligand]
        Fps_Decoy  = [Chem.GetMorganFingerprintAsBitVect(x, 2, nBits=1024) for x in Suppl_Decoy]
    else:
        Fps_Ligand = [Chem.GetMorganFingerprintAsBitVect(x, 3, useFeatures=True, nBits=1024) for x in Suppl_Ligand]
        Fps_Decoy  = [Chem.GetMorganFingerprintAsBitVect(x, 3, useFeatures=True, nBits=1024) for x in Suppl_Decoy]
        
    def sims_LL(t,r): 
        sims=DataStructs.FingerprintSimilarity(Fps_Ligand[t],Fps_Ligand[r])
        return sims
    def sims_LD(t,r): 
        sims=DataStructs.FingerprintSimilarity(Fps_Ligand[t],Fps_Decoy[r])
        return sims

    def Get_sims(k):
        Sims = []
        
        for i in range(0,length_Ligand):
            if i != k:
                Sims.append(1)
                Sims.append(sims_LL(k,i))    
        for j in range(0,length_Decoy):
            if not ((39*k<=j)&(j<39*(k+1))): 
                Sims.append(0)
                Sims.append(sims_LD(k,j))   
        return Sims

    def Get_Roc_Arg_sims(k):
        Sims_list = Get_sims(k)                                         
        data = np.array(Sims_list).reshape(-1,2)                      
        df_data = pd.DataFrame(data,columns = ['sort','sims'])           
        df = df_data.sort_values(by="sims" , ascending=False)        
        sort = list(df['sort'])
        sims = list(df['sims'])

        fpr, tpr, thresholds  =  roc_curve(sort, sims)
        roc_auc = auc(fpr,tpr)

        return (fpr, tpr,roc_auc)

    Fpr_list_sims = []
    Tpr_list_sims = []
    Auc_list_sims = []

    for k in range(0,length_Ligand):
        Roc_Arg = Get_Roc_Arg_sims(k)  
        Fpr_list_sims.append(Roc_Arg[0])
        Tpr_list_sims.append(Roc_Arg[1])
        Auc_list_sims.append(Roc_Arg[2])

    return Auc_list_sims

def set_box_color(bp, color):
    for patch in bp['boxes']:
        patch.set_facecolor(color)
    plt.setp(bp['fliers'], markerfacecolor="black")
    plt.setp(bp['medians'], color='black')

lbvs_dir = os.path.join(os.getcwd(), "datasets_ext_val_SI_classical_VS", "LBVS")
cases = ["HIVRT", "HSP90A", "ESR1", "ESR2", "FAK1"]
deepcoy_ligand_dir = [os.path.join(it, "deepcoy/actives.smi") for it in os.scandir(lbvs_dir)]
deepcoy_decoy_dir = [os.path.join(it, "deepcoy/decoys.smi") for it in os.scandir(lbvs_dir)]
tocodecoy_ligand_dir = [os.path.join(it, "tocodecoy/actives.smi") for it in os.scandir(lbvs_dir)]
tocodecoy_decoy_dir = [os.path.join(it, "tocodecoy/decoys.smi") for it in os.scandir(lbvs_dir)]

ecfp4_deepcoy = []
fcfp6_deepcoy = []
ecfp4_tocodecoy = []
fcfp6_tocodecoy = []
for case in cases:
    for dir in deepcoy_ligand_dir:
        if case in dir:
            single_deepcoy_ligands = dir
    for dir in deepcoy_decoy_dir:
        if case in dir:
            single_deepcoy_decoys = dir
    for dir in tocodecoy_ligand_dir:
        if case in dir:
            single_tocodecoy_ligands = dir
    for dir in tocodecoy_decoy_dir:
        if case in dir:
            single_tocodecoy_decoys = dir
            
    ecfp4_deepcoy.append(Compute_RocAuc(single_deepcoy_ligands, single_deepcoy_decoys))
    ecfp4_tocodecoy.append(Compute_RocAuc(single_tocodecoy_ligands, single_tocodecoy_decoys))

    fcfp6_deepcoy.append(Compute_RocAuc(single_deepcoy_ligands, single_deepcoy_decoys, "FCFP6"))
    fcfp6_tocodecoy.append(Compute_RocAuc(single_tocodecoy_ligands, single_tocodecoy_decoys, "FCFP6"))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(28, 7))
#ECFP4
line1 = ax1.axhline(y=0.5, ls=(0, (5,2.5)), c="black", linewidth=2.5)

ecfp4_deepcoy_bp = ax1.boxplot(ecfp4_deepcoy, positions=np.array(range(len(ecfp4_deepcoy)))*3.6-0.8, sym='k.', widths=0.6, patch_artist=True,)
ecfp4_tocodecoy_bp = ax1.boxplot(ecfp4_tocodecoy, positions=np.array(range(len(ecfp4_tocodecoy)))*3.6-0.2, sym='k.', widths=0.6,patch_artist=True,)

set_box_color(ecfp4_deepcoy_bp, 'darkorange') 
set_box_color(ecfp4_tocodecoy_bp, 'slategray')

ax1.set_xticks([-0.5, 3.1, 6.7, 10.3, 13.9])
ax1.set_xticklabels(cases)
ax1.set_yticks([0.2,.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
ax1.yaxis.set_major_formatter(plticker.FormatStrFormatter('%.2f'))
ax1.set_xlim(-2, len(cases)*3.08)
ax1.set_ylim(0.2, 1.0)
ax1.tick_params(labelsize=26)
ax1.set_ylabel("AUC", size="26")
ax1.set_xlabel("Case", size="26")
ax1.set_title("ECFP_4", size="26")

#FCFP6
line2 = ax2.axhline(y=0.5, ls=(0, (5,2.5)), c="black", linewidth=2.5)

fcfp6_deepcoy_bp = ax2.boxplot(fcfp6_deepcoy, positions=np.array(range(len(fcfp6_deepcoy)))*3.6-0.8, sym='k.', widths=0.6, patch_artist=True,)
fcfp6_tocodecoy_bp = ax2.boxplot(fcfp6_tocodecoy, positions=np.array(range(len(fcfp6_tocodecoy)))*3.6-0.2, sym='k.', widths=0.6,patch_artist=True,)

set_box_color(fcfp6_deepcoy_bp, 'darkorange') 
set_box_color(fcfp6_tocodecoy_bp, 'slategray')


ax2.set_xticks([-0.5, 3.1, 6.7, 10.3, 13.9])
ax2.set_xticklabels(cases)
ax2.set_yticks([0.2,.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
ax2.yaxis.set_major_formatter(plticker.FormatStrFormatter('%.2f'))
ax2.set_xlim(-2, len(cases)*3.08)
ax2.set_ylim(0.2, 1.0)
ax2.tick_params(labelsize=26)
ax2.set_xlabel("Case", size="26")
ax2.set_title("FCFP_6", size="26")

labels = ['Random distribution', 'DeepCoy', 'TocoDecoy']
fig.legend((line2, fcfp6_deepcoy_bp["boxes"][0], fcfp6_tocodecoy_bp["boxes"][0]), labels)
sns.move_legend(fig, "upper center",bbox_to_anchor=(.5, 1.1), ncol=5, 
                title=None, frameon=False, prop={"size":26}, markerscale=3)
fig.savefig("LBVS_SI.pdf", transparent=True, bbox_inches='tight')

In [ ]:
#SBVS
def read_score(file_path):
    with open(file_path) as f:
        lines = f.readlines()
 
        content = [line.rstrip() for line in lines]
        dic = {}
        
        for i, con_ in enumerate(content):
            if ('CHEMBL' in con_) or ('ZINC' in con_) or ('active' in con_) or ('decoy' in con_):
                if (con_ not in dic):
                    dic[con_] = 9999
                idx = con_

            if con_ == '> <minimizedAffinity>':
                score = float(content[i+1])
                if score < dic[idx]:
                    dic[idx] = score

    Dock = []
    if file_path.name.count('actives'):
        for val in dic.values():
            Dock.append(1)
            Dock.append(val)
    else:
        for val in dic.values():
            Dock.append(0)
            Dock.append(val)

    return Dock

def get_args(Dock_l):
    data = np.array(Dock_l).reshape(-1,2)
    df_data = pd.DataFrame(data,columns = ['type','dock'])
    df = df_data.sort_values(by="dock" , ascending=True)
    sort = list(df['type'])
    dock = list(df['dock'])

    fpr, tpr, thresholds  =  roc_curve(sort, dock, pos_label=0)
    roc_auc = auc(fpr,tpr)

    return fpr,tpr,roc_auc

sbvs_dir = os.path.join(os.getcwd(), "datasets_ext_val_SI_classical_VS", "SBVS")
cases = ["HIVRT", "HSP90A", "ESR1", "ESR2", "FAK1"]
fig, axes = plt.subplots(1, 5, figsize=(28, 7))

for i, case in enumerate(cases):
    for it in os.scandir(sbvs_dir):
        if case in it.name:
            case_dir = it
    for sdf in os.scandir(case_dir):
        if sdf.name == "smina_out_actives_deepcoy.sdf":
            deepcoy_actives_dir = sdf
        elif sdf.name == "smina_out_decoys_deepcoy.sdf":            
            deepcoy_decoys_dir = sdf
        elif sdf.name == "smina_out_actives_tocodecoy.sdf":            
            tocodecoy_actives_dir = sdf
        else:
            tocodecoy_decoys_dir = sdf
    deepcoy_actives_scores = read_score(deepcoy_actives_dir)
    deepcoy_decoys_scores = read_score(deepcoy_decoys_dir)
    deepcoy_actives_scores.extend(deepcoy_decoys_scores)
    fpr_deepcoy, tpr_deepcoy, roc_auc_deepcoy = get_args(deepcoy_actives_scores)
    print("DeepCoy")
    print(f'{case}: {roc_auc_deepcoy}')

    tocodecoy_actives_scores = read_score(tocodecoy_actives_dir)
    tocodecoy_decoys_scores = read_score(tocodecoy_decoys_dir)
    tocodecoy_actives_scores.extend(tocodecoy_decoys_scores)
    fpr_tocodecoy, tpr_tocodecoy, roc_auc_tocodecoy = get_args(tocodecoy_actives_scores)
    print("TocoDecoy")
    print(f'{case}: {roc_auc_tocodecoy}')

    axes[i].set_title(case, fontsize=26)  
    axes[i].set_xlim(0.0, 1.0)
    axes[i].set_ylim(0.0, 1.05)
    line1 = axes[i].plot([0, 1], [0, 1], color=('k'), lw=2.5, ls=(0, (5,2.5)))
    smina_deepcoy = axes[i].plot(fpr_deepcoy, tpr_deepcoy, color='darkorange', lw=3)
    smina_tocodecoy = axes[i].plot(fpr_tocodecoy, tpr_tocodecoy, color='slategray', lw=3)
    axes[i].set_xticks([0.0,.2,0.4,0.6,0.8,1.0])  
    axes[i].set_yticks([0.0,.2,0.4,0.6,0.8,1.0])
    plt.setp(axes[i].get_xticklabels(), rotation=45)
    axes[i].tick_params(labelsize=26)
    
axes[0].set_ylabel("True positive rate", size=26)
axes[2].set_xlabel("False positive rate", size=26)

labels = ['Random distribution', "DeepCoy", "TocoDecoy"]
fig.legend((line1[0], smina_deepcoy[0], smina_tocodecoy[0]), labels)
sns.move_legend(fig, "upper center",bbox_to_anchor=(.5, 0.9), ncol=5, 
                title=None, frameon=False, prop={"size":26}, markerscale=3)

for i in range(5):
    box = axes[i].get_position()
    axes[i].set_position([box.x0, box.y0, box.width, box.height*0.7])
fig.savefig("SBVS_SI.pdf", transparent=True, bbox_inches='tight')